1. The following code is for getting all the restaurant url on openrice

In [ ]:
import time
import uuid
from datetime import date, datetime

from selenium import webdriver
import re
import random
import os
import stem.process
import json
from fake_useragent import UserAgent
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options

today = str(date.today()).replace("-", "")

"""--------------------------------------------------------Setting Up TOR ----------------------------------------------------------------"""

SOCKS_PORT = 9050
TOR_PATH = os.path.normpath(r"C:\Program Files\Tor-master\tor\tor.exe")
tor_process = stem.process.launch_tor_with_config(
    config={
        "SocksPort": str(SOCKS_PORT),
        "MaxCircuitDirtiness": "1",
    },
    init_msg_handler=lambda line: (
        print(line) if re.search("Bootstrapped", line) else False
    ),
    tor_cmd=TOR_PATH,
)

PROXIES = {"http": "socks5://127.0.0.1:9050", "https": "socks5://127.0.0.1:9050"}

url = "http://ip-api.com/json/"
ua = UserAgent()
header = ua.random
headers = {"User-Agent": header}
response = requests.get(url, proxies=PROXIES, headers=headers)
result = json.loads(response.content)
print(
    "TOR IP [%s]: %s %s"
    % (datetime.now().strftime("%d-%m-%Y %H:%M:%S"), result["query"], result["country"])
)

# tor_process.kill()

"""----------------------------------------------------------------------------------------------------------------------"""

df_district = pd.read_csv(
    r"C:\Users\crmhk\Desktop\Henry\OpenRice\OpenRice_All_Districts.csv"
)

district_list = df_district["District"].tolist()

len(district_list)

PROXIES = "socks5://127.0.0.1:9050"

options = Options()
# options.add_argument("--headless=new")
options.add_argument(f"--proxy-server={PROXIES}")
link_prefix = "https://www.openrice.com/"

# Now create a blank dataframe with column hearders of district_link, restaurant_link, and district and each begin with an empty list


def scrape_resto_links_by_area(district_list):

    df_restaurant_link = pd.DataFrame(
        {"district_link": [], "restaurant_link": [], "district": []}
    )

    district_link = []
    restaurant_name_list = []
    restaurant_link = []
    district_name = []

    for district in district_list[0:]:
        url = rf"https://www.openrice.com/zh/hongkong/restaurants/district/{district}?sortBy=ORScoreDesc"
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)
        # click_button = driver.find_element(
        #     By.CLASS_NAME, "cookies-agreement-btn filled-button or-theme-y"
        # )
        # click_button.click()
        window_size = driver.get_window_size()
        window_width = window_size["width"]
        window_height = window_size["height"]
        prev_height = -1
        max_scrolls = 10
        scroll_count = 0

        def get_safe_random_offset(max_value):
            return random.randint(0, max_value - 1)

        offset_x = get_safe_random_offset(133)
        offset_y = get_safe_random_offset(133)

        while scroll_count < max_scrolls:
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            new_height = (
                driver.execute_script("return document.body.scrollHeight") - 1000
            )
            # print(height_to_scroll)
            driver.execute_script(f"window.scrollTo(0, {new_height});")
            # After scrolling down to the bottom of the page, randonly move the mouse to a different location to avoid detection
            time.sleep(random.uniform(3, 5))
            try:
                action = webdriver.ActionChains(driver)
                action.move_by_offset(offset_x, offset_y).perform()
            except:
                pass

            try:
                time.sleep(5)  # give some time for new results to load
                # new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == prev_height:
                    break
                prev_height = new_height
                scroll_count += 1
            except:
                print(
                    "----------You have scrolled to the bottom of the page!-----------"
                )
                break

        # Fetch the data using BeautifulSoup after all data is loaded
        soup = BeautifulSoup(driver.page_source, "html.parser")
        all_restaurants = soup.find_all("div", class_="poi-list-cell-desktop-right-top")
        for restaurant in all_restaurants:
            link = link_prefix + restaurant.find("a").get("href")
            restaurant_name = restaurant.find(
                "div", class_="poi-name poi-list-cell-link"
            ).text.strip()
            district_link.append(url)
            restaurant_link.append(link)
            district_name.append(district)
            restaurant_name_list.append(restaurant_name)

        driver.quit
        time.sleep(3)

    df_restaurant_link["district_link"] = district_link
    df_restaurant_link["restaurant_link"] = restaurant_link
    df_restaurant_link["district"] = district_name
    df_restaurant_link["restaurant_name"] = restaurant_name_list

    print("Done")

    return df_restaurant_link


2. The following code is for getting to basic demographic information of all the restaurants in the openrice

In [ ]:
import json
import os
import random
import re
import time
import uuid
from datetime import date, datetime

import numpy as np
import pandas as pd
import requests
import stem.process
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

today = str(date.today()).replace("-", "")

"""--------------------------------------------------------Setting Up TOR ----------------------------------------------------------------"""

SOCKS_PORT = 9050
TOR_PATH = os.path.normpath(r"<Tor Path>")
tor_process = stem.process.launch_tor_with_config(
    config={
        "SocksPort": str(SOCKS_PORT),
        "MaxCircuitDirtiness": "1",
    },
    init_msg_handler=lambda line: (
        print(line) if re.search("Bootstrapped", line) else False
    ),
    tor_cmd=TOR_PATH,
)

PROXIES = {"http": "socks5://127.0.0.1:9050", "https": "socks5://127.0.0.1:9050"}

url = "http://ip-api.com/json/"
ua = UserAgent()
header = ua.random
headers = {"User-Agent": header}
response = requests.get(url, proxies=PROXIES, headers=headers)
result = json.loads(response.content)
print(
    "TOR IP [%s]: %s %s"
    % (datetime.now().strftime("%d-%m-%Y %H:%M:%S"), result["query"], result["country"])
)

# tor_process.kill()

"""----------------------------------------------------------------------------------------------------------------------"""

scraping_status = []
restaurant_url = []
restaurant_name = []
restaurant_being_saved_times = []
restaurant_district = []
restaurant_address = []
restaurant_mtr_exit_or_mins_walk = []
restaurant_branch_num = []
restaurant_opening_hours = []
restaurant_payment_method = []
scraped_num_restaurants = 0

df_restaurant_link = pd.read_json(rf"<9000 restaurant link path>")

for restaurant_link in list(df_restaurant_link["restaurant_link"])[0:]:

    scraped_num_restaurants += 1

    url = str(restaurant_link)

    PROXIES = "socks5://127.0.0.1:9050"

    def get_safe_random_offset(max_value):
        return random.randint(0, max_value - 1)

    link_prefix = "https://www.openrice.com/"
    offset_x = get_safe_random_offset(133)
    offset_y = get_safe_random_offset(133)

    options = Options()
    # options.add_argument("--headless=new")
    options.add_argument(f"--proxy-server={PROXIES}")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        time.sleep(random.uniform(4, 8))
        driver.execute_script(f"window.scrollBy(0, 800);")
        element = driver.find_element(
            By.CSS_SELECTOR,
            ".or-section-button.collapsed[data-toggle='collapse'][data-target='#pois-filter-expandable-features']",
        )
        element.click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        soup.find("h1").find("span", class_="name").text
    except:
        scraping_status.append("Failed to scrape, need to rescrape the url")
        restaurant_url.append(url)
        restaurant_name.append("N/A")
        restaurant_being_saved_times.append("N/A")
        restaurant_district.append("N/A")
        restaurant_address.append("N/A")
        restaurant_mtr_exit_or_mins_walk.append("N/A")
        restaurant_branch_num.append("N/A")
        restaurant_opening_hours.append("N/A")
        restaurant_payment_method.append("N/A")

    else:
        scraping_status.append("Scraped successfully")
        restaurant_url.append(url)
        restaurant_name.append(soup.find("h1").find("span", class_="name").text)
        try:
            restaurant_being_saved_times.append(
                soup.find(
                    "div", class_="header-bookmark-count js-header-bookmark-count"
                ).text
            )
        except:
            restaurant_being_saved_times.append("N/A")

        try:
            restaurant_district.append(
                soup.find("div", class_="header-poi-district dot-separator").text
            )
        except:
            restaurant_district.append("N/A")

        try:

            restaurant_address.append(
                [
                    i.text
                    for i in soup.find("div", class_="address-info-section").find_all(
                        "a"
                    )
                ]
            )
        except:
            restaurant_address.append("N/A")

        try:

            restaurant_mtr_exit_or_mins_walk.append(
                soup.find("section", class_="transport-section")
                .find("div", class_="content js-text-wrapper")
                .text
            )
        except:
            restaurant_mtr_exit_or_mins_walk.append("N/A")

        try:

            restaurant_branch_num.append(
                soup.find("section", class_="all-branches-section").text
            )
        except:
            restaurant_branch_num.append("N/A")

        try:

            restaurant_opening_hours.append(
                soup.find(
                    "section", class_="opening-hours-container has-today-opening-hour"
                ).text
            )
        except:
            restaurant_opening_hours.append("N/A")

        try:

            restaurant_payment_method.append(
                soup.find("div", class_="or-section-expandable collapse in")
                .find("div", class_="comma-tags")
                .text
            )
        except:
            restaurant_payment_method.append("N/A")

    driver.quit()
    print(f"Scraped {scraped_num_restaurants} of restaurants")

print("----------------------------Done Scraping--------------------------------")


# Now put all the lists into a dataframe
df_resto_info = pd.DataFrame(
    {
        "scraping_status": scraping_status,
        "restaurant_url": restaurant_url,
        "restaurant_name": restaurant_name,
        "restaurant_bookmarked_num": restaurant_being_saved_times,
        "restaurant_district": restaurant_district,
        "restaurant_address": restaurant_address,
        "restaurant_mtr_exit_or_mins_walk": restaurant_mtr_exit_or_mins_walk,
        "restaurant_branch_num": restaurant_branch_num,
        "restaurant_opening_hours": restaurant_opening_hours,
        "restaurant_payment_method": restaurant_payment_method,
    }
)

# Convert the result DataFrame to a dictionary
result_dict = df_resto_info.to_dict(orient="records")

# Convert the dictionary to a JSON string with indentation
restaurant_info_json = json.dumps(result_dict, indent=4)

# Write the JSON string to a file
with open(
    rf"<Your_Path>\OpenRice_Restaurant_Info_{today}.json",
    "w",
) as file:
    file.write(restaurant_info_json)


3. The following code is for getting first 5 pages of comment of each restaurant

In [ ]:
import json
import os
import random
import re
import time
import uuid
from datetime import date, datetime

import bs4
import numpy as np
import pandas as pd
import requests
import stem.process
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

today = str(date.today()).replace("-", "")


"""--------------------------------------------------------Setting Up TOR ----------------------------------------------------------------"""

SOCKS_PORT = 9050
TOR_PATH = os.path.normpath(r"C:\Program Files\Tor-master\tor\tor.exe")
tor_process = stem.process.launch_tor_with_config(
    config={
        "SocksPort": str(SOCKS_PORT),
        "MaxCircuitDirtiness": "1",
    },
    init_msg_handler=lambda line: (
        print(line) if re.search("Bootstrapped", line) else False
    ),
    tor_cmd=TOR_PATH,
)

PROXIES = {"http": "socks5://127.0.0.1:9050", "https": "socks5://127.0.0.1:9050"}

url = "http://ip-api.com/json/"
ua = UserAgent()
header = ua.random
headers = {"User-Agent": header}
response = requests.get(url, proxies=PROXIES, headers=headers)
result = json.loads(response.content)
print(
    "TOR IP [%s]: %s %s"
    % (datetime.now().strftime("%d-%m-%Y %H:%M:%S"), result["query"], result["country"])
)

# tor_process.kill()

"""----------------------------------------------------------------------------------------------------------------------"""

# soup_list = []
scraping_status_list = []
restaurant_comment_page_link_list = []
account_grade_list = []
account_link_list = []
account_name_list = []
comment_text_list = []
taste_star_list = []
env_star_list = []
service_star_list = []
hygiene_star_list = []
fair_price_star_list = []
dining_date_list = []
dining_way_list = []
price_per_person_list = []
recommended_dishes_list = []
dining_time_list = []
restaurant_link_list = []
restaurant_name_list = []
restaurant_overall_rating = []
restaurant_cuisine_type = []
restaurant_price_range = []
restaurant_smiley_smile = []
restaurant_smiley_ok = []
restaurant_smiley_cry = []
comment_views_count_list = []

df_restaurant_link = pd.read_json(rf"<your address>")


def fetch_url_with_retries(url, max_retries=5, backoff_factor=1):
    PROXIES = {"http": "socks5://127.0.0.1:9050", "https": "socks5://127.0.0.1:9050"}
    ua = UserAgent()
    header = ua.random
    headers = {"User-Agent": header}
    for attempt in range(max_retries):
        try:
            response = requests.get(url, proxies=PROXIES, headers=headers, timeout=30)
            response.raise_for_status()
            return response
        except (ReadTimeout, ConnectionError) as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(backoff_factor * (2**attempt))
            else:
                raise


num_restro_scraped = 0
link_prefix = "https://www.openrice.com/"
for restaurant_link in list(df_restaurant_link["restaurant_link"])[0:100]:
    num_restro_scraped += 1
    for page_num in range(1, 6):
        resto_comment_page_link = (
            str(restaurant_link) + r"/reviews" + rf"?page={page_num}"
        )
        try:

            response = fetch_url_with_retries(resto_comment_page_link)
            soup = bs4.BeautifulSoup(response.text, "html.parser")
            restaurant_name = soup.find("h1").find("span", class_="name").text

        except:
            account_name = "N/A"
            account_grade = "N/A"
            account_link = "N/A"
            comment = "N/A"
            comment_views_count = "N/A"
            taste_star_num = "N/A"
            env_star_num = "N/A"
            service_star_num = "N/A"
            hygiene_star_num = "N/A"
            fair_price_star_num = "N/A"
            dining_date = "N/A"
            dining_way = "N/A"
            price_per_person = "N/A"
            recommended_dishes = "N/A"
            dining_time = "N/A"
            restaurant_name = "N/A"
            resto_overall_rating = "N/A"
            resto_cuisine_type = "N/A"
            resto_price_range = "N/A"
            smiley_smile = "N/A"
            smiley_ok = "N/A"
            smiley_cry = "N/A"
            restaurant_comment_page_link_list.append(resto_comment_page_link)
            account_name_list.append(account_name)
            account_grade_list.append(account_grade)
            account_link_list.append(account_link)
            comment_text_list.append(comment)
            comment_views_count_list.append(comment_views_count)
            taste_star_list.append(taste_star_num)
            env_star_list.append(env_star_num)
            service_star_list.append(service_star_num)
            hygiene_star_list.append(hygiene_star_num)
            fair_price_star_list.append(fair_price_star_num)
            dining_date_list.append(dining_date)
            dining_way_list.append(dining_way)
            price_per_person_list.append(price_per_person)
            recommended_dishes_list.append(recommended_dishes)
            dining_time_list.append(dining_time)
            restaurant_link_list.append(restaurant_link)
            restaurant_name_list.append(restaurant_name)
            restaurant_overall_rating.append(resto_overall_rating)
            restaurant_cuisine_type.append(resto_cuisine_type)
            restaurant_price_range.append(resto_price_range)
            restaurant_smiley_smile.append(smiley_smile)
            restaurant_smiley_ok.append(smiley_ok)
            restaurant_smiley_cry.append(smiley_cry)
            scraping_status_list.append("Failed, need to re-scrape the page")

        else:

            soup = bs4.BeautifulSoup(response.text, "html.parser")

            restaurant_name = soup.find("h1").find("span", class_="name").text
            if soup.find("div", class_="header-score-show-more"):
                resto_overall_rating = soup.find(
                    "div", class_="header-score-show-more"
                ).text

            if soup.find("div", class_="header-poi-price dot-separator"):
                resto_price_range = soup.find(
                    "div", class_="header-poi-price dot-separator"
                ).text

            if soup.find("div", class_="header-poi-categories dot-separator"):
                resto_cuisine_type = soup.find(
                    "div", class_="header-poi-categories dot-separator"
                ).text

            if soup.find("div", class_="header-smile-section").find_all(
                "div", class_="score-div"
            ):
                smiley_smile = (
                    soup.find("div", class_="header-smile-section")
                    .find_all("div", class_="score-div")[0]
                    .text
                )
                smiley_ok = (
                    soup.find("div", class_="header-smile-section")
                    .find_all("div", class_="score-div")[1]
                    .text
                )
                smiley_cry = (
                    soup.find("div", class_="header-smile-section")
                    .find_all("div", class_="score-div")[2]
                    .text
                )

            all_containers = soup.find_all(
                "div",
                class_="sr2-review-list-container full clearfix js-sr2-review-list-container",
            )

            for contain in all_containers:

                left_col = contain.find("div", class_="left-col")
                account_name = left_col.find("div", class_="name").text
                account_grade = left_col.find("div", class_="grade-name").text
                account_link = link_prefix + left_col.find(
                    "a", class_="or-user-avatar"
                ).get("href")

                comment = (
                    contain.find(
                        "div", class_="tab-pane content content-full js-content-full"
                    )
                    .find("section", class_="review-container")
                    .text
                )
                if contain.find("span", class_="view-count"):
                    comment_views_count = contain.find("span", class_="view-count").text

                right_col = contain.find("div", class_="right-col")

                taste_star_num = "N/A"
                env_star_num = "N/A"
                service_star_num = "N/A"
                hygiene_star_num = "N/A"
                fair_price_star_num = "N/A"

                # Iterate through the elements
                for i in right_col.find_all("div", class_="subject"):
                    if "味道" in i.text:
                        taste_star_num = len(
                            i.find_all(
                                "span",
                                class_="or-sprite-inline-block common_yellowstar_desktop",
                            )
                        )
                    if "環境" in i.text:
                        env_star_num = len(
                            i.find_all(
                                "span",
                                class_="or-sprite-inline-block common_yellowstar_desktop",
                            )
                        )
                    if "服務" in i.text:
                        service_star_num = len(
                            i.find_all(
                                "span",
                                class_="or-sprite-inline-block common_yellowstar_desktop",
                            )
                        )
                    if "衛生" in i.text:
                        hygiene_star_num = len(
                            i.find_all(
                                "span",
                                class_="or-sprite-inline-block common_yellowstar_desktop",
                            )
                        )
                    if "抵食" in i.text:
                        fair_price_star_num = len(
                            i.find_all(
                                "span",
                                class_="or-sprite-inline-block common_yellowstar_desktop",
                            )
                        )

                dining_date = "N/A"
                dining_time = "N/A"
                dining_way = "N/A"
                price_per_person = "N/A"
                recommended_dishes = "N/A"

                # Iterate through the elements
                other_dining_info = right_col.find_all(
                    "section", class_="info-section detail"
                )
                for i in other_dining_info:
                    if "用餐日期" in i.text:
                        dining_date_element = i.find("div", class_="date text")
                        if dining_date_element:
                            dining_date = dining_date_element.text
                    if "用餐時段" in i.text:
                        dining_time_element = i.find("div", class_="price text")
                        if dining_time_element:
                            dining_time = dining_time_element.text
                    if "用餐途徑" in i.text:
                        dining_way_element = i.find("div", class_="text")
                        if dining_way_element:
                            dining_way = dining_way_element.text
                    if "人均消費" in i.text:
                        price_per_person_element = i.find("div", class_="price text")
                        if price_per_person_element:
                            price_per_person = price_per_person_element.text
                    if "推介美食" in i.text:
                        recommended_dishes_element = i.find(
                            "div", class_="list recommend-dish-name-list"
                        )
                        if recommended_dishes_element:
                            recommended_dishes = recommended_dishes_element.text

                # Now append all the data to the respective lists
                restaurant_comment_page_link_list.append(resto_comment_page_link)
                account_name_list.append(account_name)
                account_grade_list.append(account_grade)
                account_link_list.append(account_link)
                comment_text_list.append(comment)
                comment_views_count_list.append(comment_views_count)
                taste_star_list.append(taste_star_num)
                env_star_list.append(env_star_num)
                service_star_list.append(service_star_num)
                hygiene_star_list.append(hygiene_star_num)
                fair_price_star_list.append(fair_price_star_num)
                dining_date_list.append(dining_date)
                dining_way_list.append(dining_way)
                price_per_person_list.append(price_per_person)
                recommended_dishes_list.append(recommended_dishes)
                dining_time_list.append(dining_time)
                restaurant_link_list.append(restaurant_link)
                restaurant_name_list.append(restaurant_name)
                restaurant_overall_rating.append(resto_overall_rating)
                restaurant_cuisine_type.append(resto_cuisine_type)
                restaurant_price_range.append(resto_price_range)
                restaurant_smiley_smile.append(smiley_smile)
                restaurant_smiley_ok.append(smiley_ok)
                restaurant_smiley_cry.append(smiley_cry)
                scraping_status_list.append("Success")

            # soup_list.append(str(soup))
        time.sleep(random.randint(3, 5))
    print(f"-------------Scraped {num_restro_scraped} restaurants-------------")


# Now put all the lists into a dataframe
df_resto_comments = pd.DataFrame(
    {
        "scraping_status": scraping_status_list,
        "restaurant_name": restaurant_name_list,
        "restaurant_link": restaurant_link_list,
        "restaurant_comment_page_link": restaurant_comment_page_link_list,
        "restaurant_overall_rating": restaurant_overall_rating,
        "restaurant_cuisine_type": restaurant_cuisine_type,
        "restaurant_price_range": restaurant_price_range,
        "restaurant_smiley_smile": restaurant_smiley_smile,
        "restaurant_smiley_ok": restaurant_smiley_ok,
        "restaurant_smiley_cry": restaurant_smiley_cry,
        "comment_account_name": account_name_list,
        "comment_account_grade": account_grade_list,
        "comment_account_link": account_link_list,
        "comment_text": comment_text_list,
        "comment_views_count": comment_views_count_list,
        "comment_taste_star": taste_star_list,
        "comment_env_star": env_star_list,
        "comment_service_star": service_star_list,
        "comment_hygiene_star": hygiene_star_list,
        "comment_fair_price_star": fair_price_star_list,
        "comment_dining_date": dining_date_list,
        "comment_dining_time": dining_time_list,
        "comment_dining_way": dining_way_list,
        "comment_price_per_person": price_per_person_list,
        "comment_recommended_dishes": recommended_dishes_list,
        # "soup": soup_list,
    }
)

df_resto_comments.describe()

# Convert the data type of column soup to string
# df_resto_comments["soup"] = df_resto_comments["soup"].astype(str)

# Convert the result DataFrame to a dictionary
result_dict = df_resto_comments.to_dict(orient="records")

# Convert the dictionary to a JSON string with indentation
restaurant_link_json = json.dumps(result_dict, indent=4)

# Write the JSON string to a file
with open(
    rf"<your address>.json",
    "w",
) as file:
    file.write(restaurant_link_json)

print("----------------Done-------------------")


# ----------------------------------------------------------------------


4. The following code is for getting the user information

In [ ]:
import json
import os
import random
import re
import time
import uuid
from datetime import date, datetime

import bs4
import numpy as np
import pandas as pd
import requests
import stem.process
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

today = str(date.today()).replace("-", "")


"""--------------------------------------------------------Setting Up TOR ----------------------------------------------------------------"""

SOCKS_PORT = 9050
TOR_PATH = os.path.normpath(r"C:\Program Files\Tor-master\tor\tor.exe")
tor_process = stem.process.launch_tor_with_config(
    config={
        "SocksPort": str(SOCKS_PORT),
        "MaxCircuitDirtiness": "1",
    },
    init_msg_handler=lambda line: (
        print(line) if re.search("Bootstrapped", line) else False
    ),
    tor_cmd=TOR_PATH,
)

PROXIES = {"http": "socks5://127.0.0.1:9050", "https": "socks5://127.0.0.1:9050"}

url = "http://ip-api.com/json/"
ua = UserAgent()
header = ua.random
headers = {"User-Agent": header}
response = requests.get(url, proxies=PROXIES, headers=headers)
result = json.loads(response.content)
print(
    "TOR IP [%s]: %s %s"
    % (datetime.now().strftime("%d-%m-%Y %H:%M:%S"), result["query"], result["country"])
)

# tor_process.kill()

"""----------------------------------------------------------------------------------------------------------------------"""

# Import json as dataFrame
df_comment_data = pd.read_json(
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\OpenRice_Restaurant_data_20241112.json"
)

df_excluded_failed = df_comment_data[
    df_comment_data["scraping_status"] != "Failed, need to re-scrape the page"
].reset_index(drop=True)


distinct_url_list_original_order = list(
    df_excluded_failed["comment_account_link"].unique()
)

len(distinct_url_list_original_order)

# Now in distinct_url_list_original_order, I want to get one url every 200 urls
every_50_urls = distinct_url_list_original_order[::50]
len(every_50_urls)


# df_top_1000_most_comment_ac = (
#     df_excluded_failed.groupby(["comment_account_name", "comment_account_link"])
#     .size()
#     .sort_values(ascending=False)
#     .head(1000)
#     .reset_index(name="count")
# )

# df_top_1000_most_comment_ac["user_id"] = df_top_1000_most_comment_ac[
#     "comment_account_link"
# ].str.extract(r"userid=(\d+).?")

df_url_every_50_url = pd.DataFrame(every_50_urls, columns=["comment_account_link"])

df_url_every_50_url["user_id"] = df_url_every_50_url[
    "comment_account_link"
].str.extract(r"userid=(\d+).?")


num_of_user_id = 0
scraping_status = []
member_name = []
member_url = []
member_level = []
member_total_num_of_comment = []
member_total_num_of_distinct_commented_resto = []
member_total_num_of_uploaded_img_vid = []
num_of_comment_in_this_url = []
comment_header = []
comment_restaurant = []
comment_restaurant_url = []
comment_over_rating_sticker = []
comment_date = []
comment_text = []
taste_rating = []
env_rating = []
service_rating = []
hygiene_rating = []
fair_price_rating = []
comment_individual_img_vid_and_its_caption = []
total_num_of_comment = "N/A"
total_num_of_distinct_commented_resto = "N/A"
total_num_of_uploaded_img_vid = "N/A"
openrice_url_prefix = "https://www.openrice.com"

for index, row in df_url_every_50_url[750:1000].iterrows():

    num_of_comment_url_in_user_ac = 0
    num_of_user_id += 1

    for num in range(1, 6):

        num_of_comment_url_in_user_ac += 1

        url = f"https://www.openrice.com/zh/gourmet/reviews.htm?userid={row['user_id']}&city=hongkong&page={num}"

        PROXIES = "socks5://127.0.0.1:9050"

        options = Options()
        # options.add_argument("--headless=new")
        options.add_argument(f"--proxy-server={PROXIES}")

        try:
            # response = requests.get(url, headers=headers, proxies=PROXIES)
            driver = webdriver.Chrome(options=options)
            driver.get(url)
            time.sleep(random.randint(3, 5))
            # Now add code to randomly scroll the page and mimic human behavior
            # url_height = random.randint(500, 1000)
            url_height = random.choice(range(500, 1001, 50))
            driver.execute_script(f"window.scrollTo(0,{url_height})")

            soup = BeautifulSoup(driver.page_source, "html.parser")
            driver.quit
            soup.find("span", id="user_username").text
            soup.find("div", id="pagingHeader").text.strip()

        except:
            scraping_status.append("Scraping Failed, need to rescrape this url")
            member_url.append("N/A")
            member_name.append("N/A")
            member_level.append("N/A")
            member_total_num_of_comment.append("N/A")
            member_total_num_of_distinct_commented_resto.append("N/A")
            member_total_num_of_uploaded_img_vid.append("N/A")
            num_of_comment_in_this_url.append("N/A")
            comment_header.append(["N/A"])
            comment_restaurant.append(["N/A"])
            comment_restaurant_url.append(["N/A"])
            comment_over_rating_sticker.append(["N/A"])
            comment_date.append(["N/A"])
            comment_text.append(["N/A"])
            taste_rating.append(["N/A"])
            env_rating.append(["N/A"])
            service_rating.append(["N/A"])
            hygiene_rating.append(["N/A"])
            fair_price_rating.append(["N/A"])
            comment_individual_img_vid_and_its_caption.append(["N/A"])
            time.sleep(random.randint(3, 5))

        else:
            # soup = bs4.BeautifulSoup(response.text, "html.parser")
            scraping_status.append("Scraping Successful")
            member_url.append(url)
            member_name.append(soup.find("span", id="user_username").text)
            member_level.append(soup.find("div", class_="txt_13 MT5").text)
            overview_tabs = soup.find_all("a", class_="myor_menu_tab a_no_underline")
            num_of_comment_in_this_url.append(
                soup.find("div", id="pagingHeader").text.strip()
            )
            for tab in overview_tabs:
                if (
                    tab.find("div", class_="txt-small")
                    and "食評" in tab.find("div", class_="txt-small").text
                ):
                    count_div = tab.find("div", class_="count")
                    total_num_of_comment = count_div.text.strip()
            member_total_num_of_comment.append(total_num_of_comment)

            for tab in overview_tabs:
                if (
                    tab.find("div", class_="txt-small")
                    and "我的餐廳" in tab.find("div", class_="txt-small").text
                ):
                    count_div = tab.find("div", class_="count")
                    total_num_of_distinct_commented_resto = count_div.text.strip()
            member_total_num_of_distinct_commented_resto.append(
                total_num_of_distinct_commented_resto
            )

            for tab in overview_tabs:
                if (
                    tab.find("div", class_="txt-small")
                    and "相片/影片" in tab.find("div", class_="txt-small").text
                ):
                    count_div = tab.find("div", class_="count")
                    total_num_of_uploaded_img_vid = count_div.text.strip()
            member_total_num_of_uploaded_img_vid.append(total_num_of_uploaded_img_vid)

            comment_header.append(
                [
                    title.text.strip() if title.text.strip() else "N/A"
                    for title in soup.find_all("div", class_="sr2_review_title")
                ]
            )
            comment_restaurant.append(
                [
                    restaurant.text.strip() if restaurant.text.strip() else "N/A"
                    for restaurant in soup.find_all(
                        "div", class_="pg_main txt_12 main_color2 MT5"
                    )
                ]
            )

            comment_restaurant_url.append(
                [
                    (
                        (openrice_url_prefix + restaurant.find("a").get("href"))
                        if restaurant.find("a").get("href")
                        else "N/A"
                    )
                    for restaurant in soup.find_all(
                        "div", class_="pg_main txt_12 main_color2 MT5"
                    )
                ]
            )

            comment_over_rating_sticker.append(
                [
                    overview.find("span")["class"] if overview.find("span") else "N/A"
                    for overview in soup.find_all("h1", class_="rel_pos")
                ]
            )

            comment_date.append(
                [
                    date.text.strip() if date.text.strip() else "N/A"
                    for date in soup.find_all("div", class_="FR MR5")
                ]
            )

            comment_text.append(
                [
                    text.text.strip() if text.text.strip() else "N/A"
                    for text in soup.find_all("div", class_="sr2_review_full")
                ]
            )

            taste_rating.append(
                [
                    5 - len(j.find_all("div", class_="vis_hidden"))
                    for i in soup.find_all("div", class_="FR", style="width:300px")
                    for j in i.find_all("div", class_="FL")
                    if "味道" in j.text
                ]
            )

            env_rating.append(
                [
                    5 - len(j.find_all("div", class_="vis_hidden"))
                    for i in soup.find_all("div", class_="FR", style="width:300px")
                    for j in i.find_all("div", class_="FL")
                    if "環境" in j.text
                ]
            )

            service_rating.append(
                [
                    5 - len(j.find_all("div", class_="vis_hidden"))
                    for i in soup.find_all("div", class_="FR", style="width:300px")
                    for j in i.find_all("div", class_="FL")
                    if "服務" in j.text
                ]
            )
            hygiene_rating.append(
                [
                    5 - len(j.find_all("div", class_="vis_hidden"))
                    for i in soup.find_all("div", class_="FR", style="width:300px")
                    for j in i.find_all("div", class_="FL")
                    if "衛生" in j.text
                ]
            )
            fair_price_rating.append(
                [
                    5 - len(j.find_all("div", class_="vis_hidden"))
                    for i in soup.find_all("div", class_="FR", style="width:300px")
                    for j in i.find_all("div", class_="FL")
                    if "抵食" in j.text
                ]
            )

            comment_individual_img_vid_and_its_caption.append(
                [
                    len(i.find_all("img", class_="lazyload review_img_og"))
                    for i in soup.find_all("div", class_="sr2_review_full")
                ]
            )

            time.sleep(random.randint(3, 5))

            print(
                f"---------------------Scraping {num_of_user_id} user_account and {num_of_comment_url_in_user_ac} comment page in this account -----------------------------"
            )

print("-----------------------Done-----------------------------")

# Now create a dataframe to store the results
df_final_result = pd.DataFrame(
    {
        "scraping_status": scraping_status,
        "member_name": member_name,
        "member_url": member_url,
        "member_level": member_level,
        "member_total_num_of_comment": member_total_num_of_comment,
        "member_total_num_of_distinct_commented_resto": member_total_num_of_distinct_commented_resto,
        "member_total_num_of_uploaded_img_vid": member_total_num_of_uploaded_img_vid,
        "num_of_comment_in_this_url": num_of_comment_in_this_url,
        "comment_header": comment_header,
        "comment_restaurant": comment_restaurant,
        "comment_restaurant_url": comment_restaurant_url,
        "comment_over_rating_sticker": comment_over_rating_sticker,
        "comment_date": comment_date,
        "comment_text": comment_text,
        "taste_rating": taste_rating,
        "env_rating": env_rating,
        "service_rating": service_rating,
        "hygiene_rating": hygiene_rating,
        "fair_price_rating": fair_price_rating,
        "individual_comment_img_vid_and_its_caption": comment_individual_img_vid_and_its_caption,
    }
)

result_dict = df_final_result.to_dict(orient="records")

# Convert the dictionary to a JSON string with indentation
user_info = json.dumps(result_dict, indent=4)

# Write the JSON string to a file
with open(
    rf"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\User_Info_{today}.json",
    "w",
) as file:
    file.write(user_info)



5. The following code is converting the restaurant Chinese Address to Latitude and Longitude for plotting map graph in tableau

In [ ]:
import time
import pandas as pd
from components.core import Address
from components.util import Similarity
from tqdm import tqdm

# List of JSON file paths
json_files = [
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241211[5000,5500].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_0_to_800_20241207.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_800_to_1600_20241208.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_1600_to_2400_20241208.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_2400_to_3000_20241208.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_3000_to_3800_20241209.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_3800_to_5000_20241210.json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241203 [8500,9006].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241204 [7000,8000].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241204 [8000,8500].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241205 [6500,7000].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241209 [6450,6500].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241209[6400,6450].json",
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\OneDrive_1_12-12-2024\OpenRice_Restaurant_Info_20241210[5500,6400].json",
]

# Read each JSON file into a DataFrame and store them in a list
dataframes = [pd.read_json(file) for file in json_files]

# Concatenate all DataFrames vertically
df = pd.concat(dataframes, axis=0)

# Reset the index of the concatenated DataFrame
df.reset_index(drop=True, inplace=True)


df = df[
    df["scraping_status"] != "Failed to scrape, need to rescrape the url"
].reset_index(drop=True)

df["restaurant_address"] = (
    df["restaurant_address"]
    .astype(str)
    .str.replace("\\n", "")
    .str.replace(",", "")
    .str.replace("'", "")
    .str.replace(" ", "")
    .str.replace("[", "")
    .str.replace("]", "")
    .str.strip()
)

address_list = df["restaurant_address"].tolist()
len(address_list)

lat = []
lon = []
region = []
district = []
street = []
estate = []
num = 0
for i in address_list[0:]:
    num += 1
    address = Address(i)
    result = address.ParseAddress()
    lat.append(result["geo"].get("Latitude"))
    lon.append(result["geo"].get("Longitude"))
    region.append(result.get("chi", "N/A").get("Region", "N/A"))
    district.append(result.get("chi", "N/A").get("ChiDistrict", "N/A"))
    street.append(result.get("chi", "N/A").get("ChiStreet", "N/A"))
    estate.append(result.get("chi", "N/A").get("ChiEstate", "N/A"))
    if num % 200 == 0:
        print(num)
        time.sleep(1)

df["Latitude"] = lat
df["Longitude"] = lon
df["Region"] = region
df["District"] = district
df["Street"] = street
df["Estate"] = estate

# get StreetName from element of street
df["Exact_Street"] = df["Street"].apply(
    lambda x: x.get("StreetName", "N/A") if x != "N/A" else "N/A"
)
df["Exact_District"] = df["District"].apply(
    lambda x: x.get("DcDistrict", "N/A") if x != "N/A" else "N/A"
)


df_resto_lat_lon = df[
    [
        "restaurant_name",
        "restaurant_url",
        "restaurant_address",
        "Latitude",
        "Longitude",
        "Region",
        "District",
        "Street",
        "Estate",
        "StreetName",
        "Exact_District",
    ]
]

df_resto_lat_lon.to_csv(
    r"C:\Users\ngaih\OneDrive - The Chinese University of Hong Kong\CUHK\Courses\STAT 5106 (Programming Techniques)\Openrice\Restaurant_Info\resto_lat_lon_20241212.csv",
    sep=";",
    index=False,
)

